<h1>Segmenting and Clustering Neighborhoods in Toronto City</h1>

#### Import the required libraries before start working on the project

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


<h4> Install the necessary packages to run read_html </h4>

In [2]:
%pip install lxml
%pip install html5lib
import html5lib

print('Libraries imported for read_html.')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Libraries imported for read_html.


<h4>Using read_html, import the data and scrap to satisfy the requirements </h4>

In [3]:
#importing the data to pandas dataframes using read_html
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
table[0].columns = table[0].iloc[0]
df_1 = table[0]

# First requirement : The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df_1.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_1 = df_1.iloc[1:]

# Second requirement : Only process the cells that have an assigned borough. 
#Ignore cells with a borough that is Not assigned.
df_1 = df_1[df_1.Borough != 'Not assigned']

# Third requirement : If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_1.Neighborhood.replace('Not assigned',df_1.Borough,inplace=True)

# Fourth requirement : More than one neighborhood can exist in one postal code area 
df_1 = df_1.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))


df_1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h4> Importing Latitude and Longitude for each postal code before working with Foursquare API using read_csv method </h4>

In [4]:
#import latitude & longitude information using read_csv method
df_2 = pd.read_csv('Geospatial_Coordinates.csv')
#Change Postal Code to PostalCode to merge two tables
df_2.rename(columns={'Postal Code':'PostalCode'},inplace=True)
#Merging two tables to satisfy the requirements
df_final=pd.merge(df_1,df_2,on='PostalCode')
df_final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
